# Predict Album sales with MLP

In [15]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import keras as K
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend.tensorflow_backend as KTF

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)

In [145]:
data = pd.read_csv('albums.csv')
data.head()

,id,artist_id,album_title,genre,year_of_pub,num_of_tracks,num_of_sales,rolling_stone_critic,mtv_critic,music_maniac_critic
0,1,1767,Call me Cat Moneyless That Doggies,Folk,2006,11,905193,4.0,1.5,3.0
1,2,23548,Down Mare,Metal,2014,7,969122,3.0,4.0,5.0
2,3,17822,Embarrassed Hungry,Latino,2000,11,522095,2.5,1.0,2.0
3,4,19565,Standard Immediate Engineer Slovakia,Pop,2017,4,610116,1.5,2.0,4.0
4,5,24941,Decent Distance Georgian,Black Metal,2010,8,151111,4.5,2.5,1.0


In [146]:
np.shape(data)

(100000, 10)

## Data to categorical

In [147]:
data = pd.concat([data,pd.get_dummies(data.genre)],axis=1)
data.drop(['genre'],axis=1, inplace=True)

## Delete unnecesary columns

In [148]:
del data['album_title']
del data['id']
del data['artist_id']
del data['year_of_pub']

In [149]:
data.head()

,num_of_tracks,num_of_sales,rolling_stone_critic,mtv_critic,music_maniac_critic,Alternative,Ambient,Black Metal,Blues,Boy Band,...,Pop-Rock,Progressive,Punk,Rap,Retro,Rock,Techno,Trap,Unplugged,Western
0,11,905193,4.0,1.5,3.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,969122,3.0,4.0,5.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11,522095,2.5,1.0,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,610116,1.5,2.0,4.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,151111,4.5,2.5,1.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Prepare data

In [150]:
x = data.loc[:, data.columns != 'num_of_sales']
y = data["num_of_sales"]

In [151]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [174]:
np.shape(y_train)

(80000,)

## MLP

Create a model with:
* Xavier initialization (glorot uniform)
* Batch normalization between each layer
* Nadam optimizer
* Mean Absolute Error for cost

In [176]:
model = Sequential()
model.add(Dense(128, input_shape = (42, ), activation = "relu",kernel_initializer='glorot_uniform',
                bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Dense(64, activation = "relu",kernel_initializer='glorot_uniform',
                bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Dense(32, activation = "relu",kernel_initializer='glorot_uniform',
                bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Dense(1, activation="linear"))
model.compile(loss = "mae", optimizer = "nadam")

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 128)               5504      
_________________________________________________________________
batch_normalization_48 (Batc (None, 128)               512       
_________________________________________________________________
dense_70 (Dense)             (None, 64)                8256      
_________________________________________________________________
batch_normalization_49 (Batc (None, 64)                256       
_________________________________________________________________
dense_71 (Dense)             (None, 32)                2080      
_________________________________________________________________
batch_normalization_50 (Batc (None, 32)                128       
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 33        
Total para

In [177]:
model.fit(x_train, y_train, epochs = 100, validation_split = 0.2)

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 18s 288us/step - loss: 498239.0584 - val_loss: 504465.4349
Epoch 2/100
64000/64000 [==============================] - 16s 251us/step - loss: 496131.4822 - val_loss: 501564.6306
Epoch 3/100
64000/64000 [==============================] - 16s 249us/step - loss: 492625.3465 - val_loss: 497476.7041
Epoch 4/100
64000/64000 [==============================] - 16s 246us/step - loss: 487983.7811 - val_loss: 492342.9766
Epoch 5/100
64000/64000 [==============================] - 16s 246us/step - loss: 482309.9046 - val_loss: 486111.6082
Epoch 6/100
64000/64000 [==============================] - 16s 249us/step - loss: 475658.4230 - val_loss: 478528.2824
Epoch 7/100
64000/64000 [==============================] - 16s 254us/step - loss: 468104.4067 - val_loss: 470940.9236
Epoch 8/100
64000/64000 [==============================] - 16s 251us/step - loss: 459728.5322 - val_loss: 462322.7455
Epoch 

## Save weigths on disk

In [205]:
model.save_weights('modelwithoutDropout2layersRegulizer.h5') 

## Testing model

In [178]:
scores = model.evaluate(x_test, y_test, verbose=0)

In [180]:
print("costo de test: "+str(scores))

costo de test: 250876.6443


## Manual Test

In [207]:
print("real value: "+str(y_test.iloc[12]))

real value: 225722


In [206]:
print("predicted value: "+str(model.predict(np.array(x_test.iloc[12]).reshape(1,42))))

predicted value: [[398261.34]]
